# Investigating Factors Affecting Housing Rental Yields (Using ONS Data)

## Purpose

The aim of this notebook is to use various public ONS (`https://www.ons.gov.uk/`) datasets to find the effects of various factors on housing yields. I hope this will give readers (and potential landlords) a better idea of what drives rental yields and so can make better decisions in the future.

Here, I look at yields (instead of raw house prices for example) since this is comparable across regions.

I found a few datasets (crime, quality of life, property sales, ...) I think will be relevant for our problem, however I expect that we're missing some relevant features (eg unemployment, number of homeowners) however these don't seem to be available on the ONS website.

These statistics are reported by local authority so we will have a reasonable number of data points to perform inference on.

In this analysis I approximate average yields using `median(rental income)/median(house price)` instead of the actual average yield `median(rental income / house price)` since the latter requires the price and rental income for each house. This is possible to obtain (eg by scraping estate agent websites), but for simplicity I will use the approximation.

## Data
Below I list links to the ONS datasets I will be using:

Crime
- https://www.ons.gov.uk/peoplepopulationandcommunity/crimeandjustice/datasets/recordedcrimedatabycommunitysafetypartnershiparea

Private Rental Statistics
- https://www.ons.gov.uk/peoplepopulationandcommunity/housing/datasets/privaterentalmarketsummarystatisticsinengland

Median House Price
- https://www.ons.gov.uk/peoplepopulationandcommunity/housing/datasets/medianpricepaidbywardhpssadataset37

Mean House Price
- https://www.ons.gov.uk/peoplepopulationandcommunity/housing/datasets/meanpricepaidbywardhpssadataset38

Property Sales
- https://www.ons.gov.uk/peoplepopulationandcommunity/housing/datasets/numberofresidentialpropertysalesbywardhpssadataset36

Population Estimates
- https://www.ons.gov.uk/peoplepopulationandcommunity/populationandmigration/populationestimates/datasets/wardlevelmidyearpopulationestimatesexperimental

Quality of Life Estimates
- https://www.ons.gov.uk/datasets/wellbeing-local-authority/editions/time-series/versions/2

Note: the ONS has an API (https://developer.ons.gov.uk/) however these not all of these datasets are not available through this at the time of writing. Also, due issues parsing these excel files, I have manually exported the data to csvs which can be straightforwardly parsed.

In [4]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
# from ons_processing